In [67]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_TRACING_v2"] = "true"
os.environ["LANGSMITH_PROJECT"] = "tutorial_chatbot"
os.environ["LANGSMITH_API_KEY"] = ""

In [68]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env", override=True)

True

In [69]:
from langsmith import utils
utils.tracing_is_enabled()

True

In [70]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("What is openBIS and how to use it?")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.youtube.com/watch?v=Iop2WmVyZjQ', 'content': 'This webinar provides a general overview of openBIS (https://openbis.ch/) is a combined data management platform, Electronic Laboratory Notebook, and Inventory Management System. openBIS helps scientists meet requirements from funding agencies, journals, and academic institutions to publish data according to the FAIR data principles. Scientists can use openBIS to document their daily experimental or computational work, store any related experimental raw and derived data, and link everything to'}, {'url': 'https://www.uzh.ch/blog/hbz/2019/11/20/how-to-use-openbis-as-electronic-lab-notebook/?lang=en', 'content': 'openBIS is a free and open-source data manager developed by the ETH Zurich and supports the full data life cycle from project inception, through'}]


In [71]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="qwen3")

In [72]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'<think>\nOkay, the user said "hi!" I should respond in a friendly and welcoming way. Let me make sure to acknowledge their greeting and offer assistance. Maybe start with a cheerful "Hello!" to match their enthusiasm. Then, ask how I can help them today. Keep it simple and open-ended so they feel comfortable sharing what they need. Let me check if there\'s anything else I should consider. No, that should be good. Ready to respond.\n</think>\n\nHello! 😊 How can I assist you today? Let me know if you have any questions or need help with something!'

In [73]:
model_with_tools = model.bind_tools(tools)

In [74]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: <think>
Okay, the user said "Hi!" so they're probably just greeting me. I need to respond politely. Since there's no specific question or request, I don't need to use any tools here. I'll just reply with a friendly message to acknowledge their greeting. Let me make sure the response is welcoming and offers assistance if they need anything else.
</think>

Hello! How can I assist you today? 😊
ToolCalls: []


In [75]:
import re
# Clean up the response content from <think> tags
response = model_with_tools.invoke([HumanMessage(content="Hi!")])
raw = response.content
clean = re.sub(r"<think>.*?</think>", "", raw, flags=re.DOTALL).strip()

print(f"ContentString: {clean}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today? 😊
ToolCalls: []


In [76]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in SF'}, 'id': 'b2ac0639-1ce9-45d8-a00f-6a2a0b2d9505', 'type': 'tool_call'}]


## Create the agent
Now that we have defined the tools and the LLM, we can create the agent. We will be using LangGraph to construct the agent. Currently, we are using a high level interface to construct the agent, but the nice thing about LangGraph is that this high-level interface is backed by a low-level, highly controllable API in case you want to modify the agent logic.

Now, we can initialize the agent with the LLM and the tools.

Note that we are passing in the model, not model_with_tools. That is because create_react_agent will call .bind_tools for us under the hood.

In [77]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run the agent
We can now run the agent with a few queries! Note that for now, these are all stateless queries (it won't remember previous interactions). Note that the agent will return the final state at the end of the interaction (which includes any inputs, we will see later on how to get only the outputs).

First up, let's see how it responds when there's no need to call a tool:

In [78]:
from langchain_core.messages import SystemMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="hi!"), SystemMessage(content="keep it short!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='7b00d86e-55cc-4da0-b4fd-1540ec25b2bd'),
 SystemMessage(content='keep it short!', additional_kwargs={}, response_metadata={}, id='b9b11a64-cde9-456b-8230-9428de048ecc'),
 AIMessage(content='<think>\nOkay, the user said "hi!". I need to respond appropriately. Since there\'s no specific question or request, just a greeting, I should reply with a friendly greeting. No need to use any tools here. Let me make sure the response is welcoming and offers assistance. Something like "Hello! How can I assist you today?" That should cover it.\n</think>\n\nHello! How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'qwen3', 'created_at': '2025-04-30T14:26:00.714144694Z', 'done': True, 'done_reason': 'stop', 'total_duration': 540469838, 'load_duration': 19735897, 'prompt_eval_count': 177, 'prompt_eval_duration': 17392020, 'eval_count': 86, 'eval_duration': 496203430, 'message': Message(role='assista

In [79]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats openBIS?")]}
)
response["messages"]

[HumanMessage(content='whats openBIS?', additional_kwargs={}, response_metadata={}, id='ae4e282f-68c7-4c5f-8a8f-314295202db7'),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen3', 'created_at': '2025-04-30T14:26:01.859899942Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1123549622, 'load_duration': 17727760, 'prompt_eval_count': 177, 'prompt_eval_duration': 16519213, 'eval_count': 188, 'eval_duration': 1086681415, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-972bf473-2df6-4ef8-9945-73a86e468426-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'openBIS'}, 'id': 'eaa7aea0-7e9d-454c-bb53-812de56b28c6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 177, 'output_tokens': 188, 'total_tokens': 365}),
 ToolMessage(content='[{"url": "https://sis.id.ethz.ch/services/rdm/openbis.html", "content": "openBIS is a combined FAIR data management platform, Electronic Laboratory Notebook

## Streaming Messages
We've seen how the agent can be called with .invoke to get a final response. If the agent executes multiple steps, this may take a while. To show intermediate progress, we can stream back messages as they occur.

In [80]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (e07ad3db-0745-4560-b2a4-49b4d025f852)
 Call ID: e07ad3db-0745-4560-b2a4-49b4d025f852
  Args:
    query: current weather in San Francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1746023146, 'localtime': '2025-04-30 07:25'}, 'current': {'last_updated_epoch': 1746022500, 'last_updated': '2025-04-30 07:15', 'temp_c': 11.7, 'temp_f': 53.1, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 10

In [81]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

<think>
Okay, the user asked for the weather in San Francisco. I called the Tavily search function with "current weather in San Francisco" and got two results. Let me check these.

The first result is from weathershogun.com, mentioning April 30, 2025. It says sunny with day 66° and night 54°. The second link is from world-weather.info, also talking about April 2025, with similar temperatures: +52° and +63°, clear sky. 

Wait, the user didn't specify a date. The current date is October 2023, but the search results are for April 2025. That's a future date. Maybe the search tool defaulted to next year's forecast? The user might be asking for the current weather, but the results are for next year. That's a problem. 

I need to clarify. Since the results are for 2025, which is not the present, perhaps there was a misunderstanding. The user might have meant the current weather, but the search returned a forecast. Maybe the search query needs adjustment. Alternatively, the tool might not have

## Adding in memory
As mentioned earlier, this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a thread_id when invoking the agent (so it knows which thread/conversation to resume from).

In [82]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [83]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [84]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='<think>\nOkay, the user said, "hi im bob!" I need to respond appropriately. Let me check the tools provided. The only tool available is tavily_search_results_json, which is for search queries. Since the user is just greeting me and introducing themselves as Bob, there\'s no need to use the search function here. I should respond with a friendly greeting and ask how they\'re doing. No tool call is necessary in this case.\n</think>\n\nHello, Bob! How are you today? 😊', additional_kwargs={}, response_metadata={'model': 'qwen3', 'created_at': '2025-04-30T14:26:23.146792573Z', 'done': True, 'done_reason': 'stop', 'total_duration': 619481370, 'load_duration': 15752858, 'prompt_eval_count': 175, 'prompt_eval_duration': 10590130, 'eval_count': 104, 'eval_duration': 591701636, 'message': Message(role='assistant', content='<think>\nOkay, the user said, "hi im bob!" I need to respond appropriately. Let me check the tools provided. The only tool available 

In [85]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='<think>\nOkay, the user asked, "whats my name?" Let me check the conversation history. Earlier, the user introduced themselves as Bob. So the answer should be Bob. No need to use any tools here since the information is already available in the history. I\'ll just respond with their name.\n</think>\n\nYour name is Bob! 😊', additional_kwargs={}, response_metadata={'model': 'qwen3', 'created_at': '2025-04-30T14:26:23.630608634Z', 'done': True, 'done_reason': 'stop', 'total_duration': 460960761, 'load_duration': 14792800, 'prompt_eval_count': 293, 'prompt_eval_duration': 13170195, 'eval_count': 72, 'eval_duration': 417777023, 'message': Message(role='assistant', content='<think>\nOkay, the user asked, "whats my name?" Let me check the conversation history. Earlier, the user introduced themselves as Bob. So the answer should be Bob. No need to use any tools here since the information is already available in the history. I\'ll just respond with thei

If you want to start a new conversation, all you have to do is change the thread_id used

In [86]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='<think>\nOkay, the user is asking, "whats my name?" I need to figure out how to respond. Let me check the tools provided. The only tool available is tavily_search_results_json, which is for searching the web. But the user is asking for their own name, which isn\'t something that can be found through a search. The assistant doesn\'t have access to the user\'s personal information unless the user provides it. Since there\'s no function to retrieve the user\'s name, I should inform them that I don\'t have that information and ask them to provide it. I can\'t use the search tool here because it\'s not relevant. So the correct response is to prompt the user for their name.\n</think>\n\nI don\'t have access to your personal information. Could you please tell me your name?', additional_kwargs={}, response_metadata={'model': 'qwen3', 'created_at': '2025-04-30T14:26:24.680476378Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1029115092, 'loa